In [1]:
import sys
sys.path.append("../")

import os
import dotenv # type: ignore
dotenv.load_dotenv()

from sqlalchemy import create_engine # type: ignore
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors # type: ignore

import pandas as pd # type: ignore
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
%load_ext autoreload
%autoreload 2


In [2]:
df_alumnos = pd.read_csv('../data/alumnos.csv', index_col = 0).reset_index(drop=True)

In [3]:
df_alumnos.head()

,nombre,apellidos,email,estudios,especialidad,ciudad,edad,telefono,sexo,motivo_compra
0,José Manuel,Barba,lbartolome@example.net,Grado,Informatica,Cáceres,43,+34 701 827 349,Hombre,Tener mas poder adquisitivo
1,Olimpia,Galvez,trinidad39@example.com,Grado Superior,Ingenieria,Toledo,24,+34 944 75 61 47,Hombre,Mejorar mi formación
2,Clemente,Botella,armando03@example.com,Bachillerato,Letras,Sevilla,23,+34 972 69 81 29,Mujer,Tener mas poder adquisitivo
3,Trinidad,Barreda,remigio43@example.net,Grado Medio,Letras,Girona,40,+34883 010 333,Hombre,mejorar empleablidad
4,Celestina,Cano,uquesada@example.org,Grado Superior,Informatica,Toledo,40,+34900 12 67 28,Hombre,Tener mas poder adquisitivo


In [4]:
df_leads = pd.read_csv('../data/leads.csv', index_col = 0).reset_index(drop=True)

In [5]:
df_leads.head()

,nombre,apellidos,email,estudios,especialidad,ciudad,edad,telefono,sexo,motivo_compra
0,Violeta,Téllez,fulgenciomontenegro@example.org,Bachillerato,Letras,Burgos,31,+34 803 299 559,Mujer,Demanda laboral en constante crecimiento
1,Maite,Palomino,aitana54@example.com,Bachillerato,Letras,Vizcaya,22,+34 900 476 714,Mujer,Fortalecimiento del perfil profesional frente ...
2,Miriam,Almagro,odalisramon@example.org,Licenciatura,Ingenieria,Granada,24,+34 800 601 433,Hombre,Capacidad de adaptación a diferentes industrias
3,Almudena,Villaverde,graciana78@example.com,Bachillerato,Informatica,Córdoba,27,+34 739 00 32 50,Hombre,Ahorrar dinero en equipos de desarrollo para p...
4,Primitivo,Saldaña,baenacipriano@example.net,Grado Superior,Informatica,Zaragoza,30,+34841 44 87 97,Hombre,Aventurarse en el mundo freelance con más conf...


In [6]:
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
ACCOUNT_IDENTIFIER = os.getenv("ACCOUNT_IDENTIFIER")

In [7]:

user = DB_USER
password = DB_PASSWORD
conn_string = f"postgresql+psycopg2://{user}:{password}@{ACCOUNT_IDENTIFIER}:5432/{DB_NAME}"
engine = create_engine(conn_string)

In [8]:

table_name = 'alumnos'
if_exists = 'replace'
 
# Write the data to Snowflake
with engine.connect() as con:
 df_alumnos.to_sql(
 name=table_name.lower(), 
 con=con, 
 if_exists=if_exists
 )

In [9]:
table_name = 'leads'
if_exists = 'replace'
 
# Write the data to Snowflake
with engine.connect() as con:
 df_leads.to_sql(
 name=table_name.lower(), 
 con=con, 
 if_exists=if_exists
 )

In [10]:
# Conectar a la base de datos
try:
    connection = psycopg2.connect(
        database=DB_NAME, # la base de datos con la que queremos trabajar
        user=DB_USER, # el usuario que tenemos 
        password=DB_PASSWORD,
        host=ACCOUNT_IDENTIFIER)
    print("conexion realizada con éxito")

except OperationalError as e:
    # Manejo de errores comunes
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("Contraseña inválida.")
        
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión.")
    else:
        print(f"Ocurrió un error: {e}")


# Una vez que hemos creado la conexión, creamos un cursor
cursor = connection.cursor()

# Ejecutar una consulta SELECT
cursor.execute('SELECT * FROM "alumnos";')

# Recuperar los resultados
resultados_alumnos = cursor.fetchall()

print(f"resultados en la tabla alumnos: {len(resultados_alumnos)}")
# Procesar los resultados

cursor.execute('SELECT * FROM "leads";')

resultados_leads = cursor.fetchall()

print(f"resultados en la tabla leads: {len(resultados_leads)}")

# Cerrar cursor y conexión
cursor.close()
connection.close()

conexion realizada con éxito
resultados en la tabla alumnos: 9934
resultados en la tabla leads: 983


In [11]:
columnas = [desc[0] for desc in cursor.description]  
df_rec_bd_alumnos = pd.DataFrame(resultados_alumnos, columns=columnas).reset_index(drop=True)
df_rec_bd_alumnos = df_rec_bd_alumnos.drop('index', axis=1)
df_rec_bd_alumnos.head()

,nombre,apellidos,email,estudios,especialidad,ciudad,edad,telefono,sexo,motivo_compra
0,José Manuel,Barba,lbartolome@example.net,Grado,Informatica,Cáceres,43,+34 701 827 349,Hombre,Tener mas poder adquisitivo
1,Olimpia,Galvez,trinidad39@example.com,Grado Superior,Ingenieria,Toledo,24,+34 944 75 61 47,Hombre,Mejorar mi formación
2,Clemente,Botella,armando03@example.com,Bachillerato,Letras,Sevilla,23,+34 972 69 81 29,Mujer,Tener mas poder adquisitivo
3,Trinidad,Barreda,remigio43@example.net,Grado Medio,Letras,Girona,40,+34883 010 333,Hombre,mejorar empleablidad
4,Celestina,Cano,uquesada@example.org,Grado Superior,Informatica,Toledo,40,+34900 12 67 28,Hombre,Tener mas poder adquisitivo


In [12]:
columnas = [desc[0] for desc in cursor.description]  
df_rec_bd_leads = pd.DataFrame(resultados_leads, columns=columnas).reset_index(drop=True)
df_rec_bd_leads = df_rec_bd_leads.drop('index', axis=1)
df_rec_bd_leads.head()

,nombre,apellidos,email,estudios,especialidad,ciudad,edad,telefono,sexo,motivo_compra
0,Violeta,Téllez,fulgenciomontenegro@example.org,Bachillerato,Letras,Burgos,31,+34 803 299 559,Mujer,Demanda laboral en constante crecimiento
1,Maite,Palomino,aitana54@example.com,Bachillerato,Letras,Vizcaya,22,+34 900 476 714,Mujer,Fortalecimiento del perfil profesional frente ...
2,Miriam,Almagro,odalisramon@example.org,Licenciatura,Ingenieria,Granada,24,+34 800 601 433,Hombre,Capacidad de adaptación a diferentes industrias
3,Almudena,Villaverde,graciana78@example.com,Bachillerato,Informatica,Córdoba,27,+34 739 00 32 50,Hombre,Ahorrar dinero en equipos de desarrollo para p...
4,Primitivo,Saldaña,baenacipriano@example.net,Grado Superior,Informatica,Zaragoza,30,+34841 44 87 97,Hombre,Aventurarse en el mundo freelance con más conf...
